<a href="https://colab.research.google.com/github/Jorya777/Backup/blob/main/final_assignment_(UNSDCF_evaluation_dashboard).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit plotly pandas numpy
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb >/dev/null 2>&1 || true


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install PyPDF2 python-docx tqdm nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 21.8 MB/s eta 0:00:00


In [4]:
import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
# ==========================================================
# UNSDCF Evaluation Reports Text Extraction and Analysis
# Generates:
#   1. relevant_sentences_UNSDCF_filtered.csv
#   2. word_frequency_UNSDCF.csv
#   3. actor_cooccurrence_UNSDCF.csv
# ==========================================================

import os
import re
import pandas as pd
from nltk.tokenize import sent_tokenize
from PyPDF2 import PdfReader
from docx import Document
from tqdm import tqdm
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from itertools import combinations

# ----------------------------------------------------------
# Init
# ----------------------------------------------------------
nltk.download("punkt")
nltk.download("vader_lexicon")
nltk.download("stopwords")

DATA_DIR = "/content/drive/MyDrive/evaluation_reports"
OUTPUT_SENT = "/content/drive/MyDrive/relevant_sentences_UNSDCF_filtered.csv"
OUTPUT_WORD = "/content/drive/MyDrive/word_frequency_UNSDCF.csv"
OUTPUT_COOC = "/content/drive/MyDrive/actor_cooccurrence_UNSDCF.csv"

# ----------------------------------------------------------
# Step 1: Extract text
# ----------------------------------------------------------
def extract_text(file_path):
    text = ""
    try:
        if file_path.endswith(".pdf"):
            reader = PdfReader(file_path)
            for page in reader.pages:
                t = page.extract_text()
                if t:
                    text += t + "\n"
        elif file_path.endswith(".docx"):
            doc = Document(file_path)
            text = "\n".join([p.text for p in doc.paragraphs if p.text.strip()])
    except Exception as e:
        print(f"⚠️ Failed to read {file_path}: {e}")
    return text

# ----------------------------------------------------------
# Step 2: Extract relevant sentences
# ----------------------------------------------------------
KEYWORDS = ["DCO", "RC", "UNCT"]

def extract_relevant_sentences(text, country, filename):
    sentences = sent_tokenize(text)
    data = []
    for i, s in enumerate(sentences):
        if any(k in s for k in KEYWORDS):
            context = " ".join(sentences[max(0, i-1):min(len(sentences), i+2)])
            data.append({
                "Country": country,
                "Sentence": context.strip(),
                "SourceFile": filename
            })
    return pd.DataFrame(data)

# ----------------------------------------------------------
# Step 3: Loop through reports
# ----------------------------------------------------------
all_data = []
for file in tqdm(os.listdir(DATA_DIR)):
    if not (file.endswith(".pdf") or file.endswith(".docx")):
        continue
    file_path = os.path.join(DATA_DIR, file)
    country = re.sub(r"[^A-Za-z]", " ", os.path.splitext(file)[0]).split()[0]
    text = extract_text(file_path)
    df = extract_relevant_sentences(text, country, file)
    all_data.append(df)

df_all = pd.concat(all_data, ignore_index=True)
print(f"✅ Extracted {len(df_all)} relevant sentences.")

# ----------------------------------------------------------
# Step 4: Sentiment analysis + Actor label
# ----------------------------------------------------------
sia = SentimentIntensityAnalyzer()
df_all["Sentiment"] = df_all["Sentence"].apply(lambda x: sia.polarity_scores(x)["compound"])
df_all["Sentiment_Label"] = df_all["Sentiment"].apply(
    lambda s: "Positive" if s > 0.2 else ("Negative" if s < -0.2 else "Neutral")
)

def detect_actor(sentence):
    actors = []
    for a in ["DCO", "RC", "UNCT"]:
        if re.search(rf"\b{a}\b", sentence):
            actors.append(a)
    return ", ".join(actors) if actors else "Unspecified"

df_all["Actor"] = df_all["Sentence"].apply(detect_actor)
df_all = df_all[df_all["Actor"] != "Unspecified"]
df_all.to_csv(OUTPUT_SENT, index=False)
print(f"📂 Saved filtered sentences to: {OUTPUT_SENT}")

# ----------------------------------------------------------
# Step 5: Word frequency
# ----------------------------------------------------------
def clean_text(s):
    if not isinstance(s, str):
        return ""
    s = s.lower()
    s = re.sub(r"[^a-z\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

df_all["Sentence_clean"] = df_all["Sentence"].astype(str).apply(clean_text)

stop_list = list(set(stopwords.words("english")))
vectorizer = CountVectorizer(stop_words=stop_list, max_features=1000, min_df=2)
X = vectorizer.fit_transform(df_all["Sentence_clean"])

word_freq = pd.DataFrame({
    "word": vectorizer.get_feature_names_out(),
    "count": X.toarray().sum(axis=0)
}).sort_values(by="count", ascending=False)
word_freq.to_csv(OUTPUT_WORD, index=False)
print(f"✅ Saved word frequency to: {OUTPUT_WORD}")

# ----------------------------------------------------------
# Step 6: Actor co-occurrence
# ----------------------------------------------------------
cooccurrence = []
for _, row in df_all.iterrows():
    found = [a for a in ["UNCT", "RC", "DCO"] if a in row["Sentence"]]
    if len(found) > 1:
        for combo in combinations(found, 2):
            cooccurrence.append(combo)

if cooccurrence:
    df_cooc = pd.DataFrame(cooccurrence, columns=["Actor1", "Actor2"])
    df_cooc["pair"] = df_cooc.apply(lambda x: " & ".join(sorted([x["Actor1"], x["Actor2"]])), axis=1)
    df_cooc = df_cooc.groupby("pair").size().reset_index(name="count").sort_values(by="count", ascending=False)
    df_cooc.to_csv(OUTPUT_COOC, index=False)
    print(f"✅ Saved actor co-occurrence to: {OUTPUT_COOC}")
else:
    print("⚠️ No co-occurring actors found.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
100%|██████████| 6/6 [00:46<00:00,  7.82s/it]


✅ Extracted 737 relevant sentences.
📂 Saved filtered sentences to: /content/drive/MyDrive/relevant_sentences_UNSDCF_filtered.csv
✅ Saved word frequency to: /content/drive/MyDrive/word_frequency_UNSDCF.csv
✅ Saved actor co-occurrence to: /content/drive/MyDrive/actor_cooccurrence_UNSDCF.csv


In [ ]:
%%writefile /content/drive/MyDrive/app.py
import streamlit as st
import pandas as pd
import plotly.express as px

st.set_page_config(page_title="UNSDCF Evaluation Dashboard", layout="wide")

st.title("🌍 United Nations Sustainable Development Cooperation Framework Evaluation Dashboard")


file_path = "/content/drive/MyDrive/2021-2023 evaluation expenditures analysis .xlsx"
df_spend = pd.read_excel(file_path)
df_spend.columns = df_spend.columns.str.strip()
df_spend.rename(columns={
    "Evaluation expenditure($)": "Evaluation Spending ($)",
    "Program Expenditure": "Program Expenditure",
    "The proportion of Evaluation Expenditure to Program Expenditure": "Eval Ratio (%)"
}, inplace=True)
for c in ["Evaluation Spending ($)", "Program Expenditure", "Eval Ratio (%)"]:
    df_spend[c] = pd.to_numeric(df_spend[c], errors="coerce")
df_spend.dropna(subset=["Eval Ratio (%)"], inplace=True)


st.subheader("🌍 Evaluation Countries (2021–2023)")
fig_map = px.scatter_geo(df_spend, locations="Country", locationmode="country names",
                         hover_name="Country", hover_data={"Evaluation year ": True},
                         text="Evaluation year ", projection="natural earth")
st.plotly_chart(fig_map, use_container_width=True)


st.subheader("💰 Evaluation vs Programme Expenditure")
fig_scatter = px.scatter(df_spend, x="Program Expenditure", y="Eval Ratio (%)",
                         size="Evaluation Spending ($)",
                         color="Region" if "Region" in df_spend.columns else "Country",
                         hover_name="Country")
st.plotly_chart(fig_scatter, use_container_width=True)


CRITERIA = ['relevance','coherence','effectiveness','efficiency','orientation towards impact','sustainability']
countries_eval = ["Azerbaijan","Uganda","Serbia","Indonesia","Panama","Bosnia and Herzegovina"]
scores = {
    "Azerbaijan":[4,3,4,3,3,3],
    "Uganda":[4,2,4,3,3,3],
    "Serbia":[4,2,4,3,3,3],
    "Indonesia":[5,3,4,3,4,3],
    "Panama":[4,3,3,3,3,2],
    "Bosnia and Herzegovina":[4,2,4,3,3,3]
}
df_scores = pd.DataFrame([{"Country":c,"Criterion":crit,"Score":scores[c][i]}
                          for c in countries_eval for i,crit in enumerate(CRITERIA)])
country = st.sidebar.selectbox("Select Country", countries_eval)
fig_radar = px.line_polar(df_scores[df_scores["Country"]==country],
                          r="Score", theta="Criterion", line_close=True)
st.plotly_chart(fig_radar, use_container_width=True)

# ----------------------------------------------------------
# Text analysis
# ----------------------------------------------------------
st.header(" Text Analysis: Mentions of DCO / RC / UNCT")

try:
    df_mentions = pd.read_csv("/content/drive/MyDrive/relevant_sentences_UNSDCF_scored.csv")
    df_words = pd.read_csv("/content/drive/MyDrive/word_frequency_UNSDCF.csv")

    st.subheader("📑 Sample Extracted Mentions")
    st.dataframe(df_mentions.head(10))

    st.subheader("📊 Sentiment Distribution")
    sent_summary = df_mentions.groupby("Sentiment_Label").size().reset_index(name="Count")
    fig_sent = px.bar(sent_summary, x="Sentiment_Label", y="Count", color="Sentiment_Label")
    st.plotly_chart(fig_sent, use_container_width=True)

    st.subheader("🔤 Top Keywords in Mentions")
    top_words = df_words.head(20)
    fig_words = px.bar(top_words, x="word", y="count", title="Top 20 Words in DCO/RC/UNCT Mentions", color="count")
    st.plotly_chart(fig_words, use_container_width=True)

except Exception as e:
    st.warning(f"⚠️ Text analysis results not found or failed to load: {e}")

st.markdown("---")
st.markdown("© United Nations DCO – Data visualization for learning purposes")


Overwriting /content/drive/MyDrive/app.py


In [ ]:
import subprocess, time, re, sys

PORT = "8501"
# 启动 Streamlit 后台服务
streamlit = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", PORT, "--server.headless", "true"],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
)
time.sleep(2)

# 启动 Cloudflare Tunnel
tunnel = subprocess.Popen(
    ["cloudflared", "tunnel", "--url", f"http://localhost:{PORT}", "--no-autoupdate"],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
)

print("Starting Cloudflare Tunnel … (keep this cell running)")
for line in tunnel.stdout:
    sys.stdout.write(line)
    sys.stdout.flush()
    m = re.search(r"https://[a-z0-9-]+\.trycloudflare\.com", line)
    if m:
        print("\n🚀 Streamlit app URL:", m.group(0))
        print("No password required. Keep this cell running while you use the app.")
        break


Starting Cloudflare Tunnel … (keep this cell running)
2025-10-25T19:31:52Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-10-25T19:31:52Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-10-25T19:31:55Z INF +--------------------------------------------------------------------------------------------+
2025-10-25T19:31:55Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-10-25T19:31:55Z INF | 